# Shenfun - High-Performance Computing platform for the Spectral Galerkin method

<center>

<div><img src="Chebyshev_Polynomials_of_the_First_Kind.svg" width="300"></div>

<div class="sl-block" style="height: auto; width: 600px;">
    <div>
        <p><center style="font-size:1.2em">Professor Mikael Mortensen</p>
        <p><center>Department of Mathematics, University of Oslo</p>
        <p><center>Presented at the International Conference on Scientific Computing and Applications (ICSCA), Xiamen, China, 29/5 - 2019</p>
    </div>
</div>

# Shenfun - facts

1. Shenfun is named in honour of Professor Jie Shen:-) 
2. Shenfun is a high performance computing platform for solving partial differential equations (PDEs) by the spectral Galerkin method (with numerical integration).
3. Shenfun has been run with 65,000 processors on a Cray XC40.
4. Shenfun is a Python package originally developed for pseudo-spectral turbulence simulations.

<img src="RB_200k_small.png" style="float:left" width="300"> <img src="isotropic_cropped.gif" style="float:right" width="200"> 
<p style="clear: both;">

# Python is a scripting language
<p style="margin-bottom:1cm;">

No compilation - just execute. Much like MATLAB.

In this presentation this is a Python terminal:

Inside the terminal any Python code can be executed. For example, import the `shenfun` module and execute `help(shenfun)` to get some information on the module:

In [1]:
import shenfun
help(shenfun)

Help on package shenfun:

NAME
    shenfun - This is the **shenfun** package

DESCRIPTION
    What is **shenfun**?
    
    ``Shenfun`` is a high performance computing platform for solving partial
    differential equations (PDEs) by the spectral Galerkin method. The user
    interface to shenfun is very similar to
    `FEniCS <https://fenicsproject.org>`_, but applications are limited to
    multidimensional tensor product grids. The code is parallelized with MPI
    through the `mpi4py-fft <https://bitbucket.org/mpi4py/mpi4py-fft>`_
    package.
    
    ``Shenfun`` enables fast development of efficient and accurate PDE solvers
    (spectral order and accuracy), in the comfortable high-level Python language.
    The spectral accuracy is ensured from using high-order *global* orthogonal
    basis functions (Fourier, Legendre and Chebyshev), as opposed to finite element
    codes like `FEniCS <https://fenicsproject.org>`_ that are using low-order
    *local* basis functions. Efficiency

# Living presentation

For your benefit I'm now in presentation mode. 


However, this presentation is written with [jupyter](https://jupyter.org/) and if you open it in active mode, then all boxes like this 

will be alive and ready to execute Python code. 

Open https://github.com/spectralDNS/shenfun/ and press the launch-binder button. Wait for binder to launch and choose the `Shenfun presentation.ipynb` file to get a live document.

https://mybinder.org/v2/gh/spectralDNS/shenfun/master?filepath=binder


# The Spectral Galerkin method


<p style="margin-bottom:1cm;">
Approximates solutions $u(x)$ using global basis functions $\phi_k(x)$

  
$$
u(x) = \sum_{k=0}^{N-1}\hat{u}_k \phi_k(x)
$$


## Multidimensional problems
<p style="margin-bottom:1cm;">

$$
u(x, y) = \sum_{k=0}^{N_0-1}\sum_{l=0}^{N_1-1}\hat{u}_{kl} \phi_{kl}(x, y)
$$

$$
u(x, y, z) = \sum_{k=0}^{N_0-1}\sum_{l=0}^{N_1-1} \sum_{m=0}^{N_2-1}\hat{u}_{klm} \phi_{klm}(x, y, z)
$$

where $\phi_{kl}(x, y)$ and $\phi_{klm}(x, y, z)$ are outer products of (possibly different) basis functions. Like

$$
\phi_{kl}(x, y) = T_k(x) L_l(y)
$$

# The Spectral Galerkin method

solves PDEs, like Poisson's equation

\begin{align}
\nabla^2 u(x) &= f(x), \quad x \in [-1, 1] \\
u(\pm 1) &= 0
\end{align}


using variational forms: find $u \in H^1_0$ such that 

$$(\nabla^2u, v)_w^N = (f, v)_w^N \quad \forall v \in H^1_0$$

where $(u, v)_w^{N}$ is a weighted inner product.

# Weighted inner products

Tthe weighted inner product is defined as

$$
 (u, v)_w = \int_{\Omega} u \overline{v} w \, \mathbf{dx},
$$

where $w(\mathbf{x})$ is a weight associated with the chosen basis (different bases have different weights), and $v(\mathbf{x})$ and $u(\mathbf{x})$ are test and trial functions, respectively. The overline represents a complex conjugate.

Note that $\Omega$ can be any tensor product domain spanned by the chosen 1D bases. For a 2D basis with Chebyshev in both directions $\Omega=[-1, 1]^2$ and $\mathbf{dx}=dxdy$. For mixed Chebyshev with Fourier it is $\Omega=[-1, 1]\times[0, 2\pi]$. 


# In Shenfun quadrature is used for the integrals

1D with Chebyshev basis:
$$
\int_{-1}^1 u v w \, {dx} \approx \sum_{i=0}^{N-1} u(x_i) v(x_i) \omega_i,
$$

where $\{\omega_i\}_{i=0}^{N-1}$ are the quadrature weights associated with the chosen basis and quadrature rule. The associated quadrature points are denoted as $\{x_i\}_{i=0}^{N-1}$. For Chebyshev we can choose between `Chebyshev-Gauss` or `Chebyshev-Gauss-Lobatto`, whereas for Legendre the choices are `Legendre-Gauss` or `Legendre-Gauss-Lobatto`. 

2D with mixed Chebyshev-Fourier:
$$
\int_{-1}^1\int_{0}^{2\pi} u \overline{v} w \, {dxdy} \approx \sum_{i=0}^{N_0-1}\sum_{j=0}^{N_1-1} u(x_i, y_j) \overline{v}(x_i, y_j) \omega^{(x)}_i \omega_j^{(y)} ,
$$


# Spectral Galerkin solution procedure

1. Choose basis function(s) satisfying correct boundary conditions
2. Transform PDEs to variational forms using the method of weighted residuals
3. Assemble variational forms and solve resulting linear algebra systems

# Basis functions


| Family    | Basis (span)                             | Domain    |
|  :---:    |         :---:                            |   :---:   |
| Chebyshev | $$\{T_k\}_{k=0}^{N-1}$$                  | $$[-1, 1]$$ |
| Legendre  | $$\{L_k\}_{k=0}^{N-1}$$                  | $$[-1, 1]$$ |
| Fourier   | $$\{\exp(\text{i}kx)\}_{k=-N/2}^{N/2-1}$$| $$[0, 2\pi]$$ |
| Hermite   | $$\{H_k\}_{k=0}^{N-1}$$                  | $$[-\infty, \infty]$$|
| Laguerre  | $$\{La_k\}_{k=0}^{N-1}$$                 | $$[0, \infty]$$ |


In [2]:
from shenfun import *
N = 8
C = Basis(N, 'Chebyshev')
L = Basis(N, 'Legendre')
x, w = C.points_and_weights()
print(np.vstack((x, w)).T)

[[ 0.98078528  0.39269908]
 [ 0.83146961  0.39269908]
 [ 0.55557023  0.39269908]
 [ 0.19509032  0.39269908]
 [-0.19509032  0.39269908]
 [-0.55557023  0.39269908]
 [-0.83146961  0.39269908]
 [-0.98078528  0.39269908]]


# Bases with non-periodic boundary conditions

## Dirichlet

| family    | Basis (span)          | Boundary condition |
|-----------|-----------------------|----------|
| Chebyshev | $$\{T_k-T_{k+2}\}_{k=0}^{N-3}$$ | $$u(\pm 1) = 0$$ |
| Legendre  | $$\{L_k-L_{k+2}\}_{k=0}^{N-3}$$ | $$u(\pm 1) = 0$$ |
| Hermite   | $$\exp(-x^2)\{H_k\}_{k=0}^{N-1}$$ | $$u(\pm \infty) = 0$$ |
| Laguerre  | $$\exp(-x/2)\{La_k-La_{k+1}\}_{k=0}^{N-2}$$| $$u(0) = u(\infty) = 0$$ |

In [3]:
C0 = Basis(N, 'Chebyshev', bc=(0, 0))
H0 = Basis(N, 'Hermite')
La = Basis(N, 'Laguerre', bc=(0, 0))

## Neumann $u'(\pm 1) = 0$

| family    | Basis (span)          |
|-----------|-----------------------|
| Chebyshev | $$\{T_k-\frac{k^2}{(k+2)^2}T_{k+2}\}_{k=0}^{N-3}$$ | 
| Legendre  | $$\{L_k-\frac{k(k+1)}{(k+2)(k+3)}L_{k+2}\}_{k=0}^{N-3}$$ |

In [4]:
CN = Basis(N, 'Chebyshev', bc='Neumann')
LN = Basis(N, 'Legendre', bc='Neumann')

## Biharmonic $u(\pm 1) = u'(\pm 1) = 0$

| family    | Basis (span)          |
|-----------|-----------------------|
| Chebyshev | $$\{T_k-\frac{2(k+2)}{k+3}T_{k+2}+\frac{k+1}{k+3} T_{k+4}\}_{k=0}^{N-5}$$ | 
| Legendre  | $$\{L_k-\frac{2(2k+5)}{(2k+7)}L_{k+2}+\frac{2k+3}{2k+7}L_{k+4}\}_{k=0}^{N-5}$$ |


In [5]:
CB = Basis(N, 'Chebyshev', bc='Biharmonic')
LB = Basis(N, 'Legendre', bc='Biharmonic')

# Multidimensional tensor product spaces


In [6]:
C0 = Basis(N, 'Chebyshev', bc=(0, 0))
C1 = Basis(N, 'Chebyshev')
CC = TensorProductSpace(comm, (C0, C1))
L1 = Basis(N, 'Legendre')
CL = TensorProductSpace(comm, (C0, C1))

In [7]:
help(CL)

Help on TensorProductSpace in module shenfun.tensorproductspace object:

class TensorProductSpace(mpi4py_fft.mpifft.PFFT)
 |  TensorProductSpace(comm, bases, axes=None, dtype=None, slab=False, collapse_fourier=False, **kw)
 |  
 |  Class for multidimensional tensorproductspaces.
 |  
 |  The tensorproductspaces are created as Cartesian products from a set of 1D
 |  bases. The 1D bases are subclassed instances of the :class:`.SpectralBase`
 |  class.
 |  
 |  Parameters
 |  ----------
 |  comm : MPI communicator
 |  bases : list
 |      List of 1D bases
 |  axes : tuple of ints, optional
 |      A tuple containing the order of which to perform transforms.
 |      Last item is transformed first. Defaults to range(len(bases))
 |  dtype : data-type, optional
 |      Type of input data in real physical space. If not provided it
 |      will be inferred from the bases.
 |  slab : bool, optional
 |      Use 1D slab decomposition.
 |  collapse_fourier : bool, optional
 |      Collapse axes for

# How about equations?

In [8]:
L0 = Basis(N, 'Legendre', bc=(0, 0))
u = TrialFunction(L0)
v = TestFunction(L0)
A = inner(grad(u), grad(v))

In [9]:
print(A)

{0: array([ 6., 10., 14., 18., 22., 26.])}


In [10]:
print(A.diags().todense())

[[ 6.  0.  0.  0.  0.  0.]
 [ 0. 10.  0.  0.  0.  0.]
 [ 0.  0. 14.  0.  0.  0.]
 [ 0.  0.  0. 18.  0.  0.]
 [ 0.  0.  0.  0. 22.  0.]
 [ 0.  0.  0.  0.  0. 26.]]


A diagonal stiffness matrix!

# 2D

In [11]:
L0 = Basis(N, 'Legendre', bc=(0, 0))
F1 = Basis(N, 'Fourier', dtype='d')
TP = TensorProductSpace(comm, (L0, F1))
u = TrialFunction(TP)
v = TestFunction(TP)
A = inner(grad(u), grad(v))

In [12]:
print(A)

[<shenfun.matrixbase.TPMatrix object at 0x11e45bf98>, <shenfun.matrixbase.TPMatrix object at 0x11e45bf28>]


# ? 

A is a list of two TPMatrix objects???


# `TPMatrix` is a Tensor Product matrix

A `TPMatrix` is the outer product of smaller matrices (2 in 2D, 3 in 3D etc).  

Consider the inner product:

$$
\begin{align}
(\nabla u, \nabla v) &= \frac{1}{2\pi}\int_{-1}^{1}\int_{0}^{2\pi} \left(\frac{\partial u}{\partial x}, \frac{\partial u}{\partial y}\right) \cdot \left(\frac{\partial \overline{v}}{\partial x}, \frac{\partial \overline{v}}{\partial y}\right) {dxdy} \\
(\nabla u, \nabla v) &= \frac{1}{2\pi}\int_{-1}^1 \int_{0}^{2\pi} \frac{\partial u}{\partial x}\frac{\partial \overline{v}}{\partial x} {dxdy} + \int_{-1}^1 \int_{0}^{2\pi} \frac{\partial u}{\partial y}\frac{\partial \overline{v}}{\partial y} {dxdy}
\end{align}
$$

which is also a sum of two terms. These two terms are the two `TPMatrix`es returned by `inner` above.

Now each one of these two terms can be written as the outer product of two smaller matrices. 

Consider the first, inserting for 

$$
\begin{align}
v &= \phi_{kl} = (L_k(x)-L_{k+2}(x))\exp(\text{i}nl) \\
u &= \sum_m \sum_n \hat{u}_{mn} \phi_{mn}
\end{align}
$$
we get
$$
\begin{align}
& \frac{1}{2\pi}\int_{-1}^1 \int_{0}^{2\pi} \frac{\partial u}{\partial x}\frac{\partial v^*}{\partial x} {dxdy} = \frac{1}{2\pi}\int_{-1}^1 \int_{0}^{2\pi} \frac{\partial \sum_{m}\sum_{n} \hat{u}_{mn} \phi_{mn}}{\partial x}\frac{\partial \phi_{kl}^*}{\partial x }{dxdy}
\end{align}
$$

$$
\begin{align}
   &= \sum_{m}\sum_{n} \hat{u}_{mn} \frac{1}{2\pi} \int_{-1}^1 \int_{0}^{2\pi} \frac{\partial (L_m-L_{m+2})\exp(iny)}{\partial x}\frac{\partial (L_k-L_{k+2})\exp(-ily)}{\partial x} {dxdy} \\
   &= \sum_{m}\sum_{n} \hat{u}_{mn} \frac{1}{2\pi} \int_{-1}^1 \int_{0}^{2\pi} \frac{\partial (L_m-L_{m+2})}{\partial x}\frac{\partial (L_k-L_{k+2})}{\partial x} \exp(iny) \exp(-ily) {dxdy} \\
   &= \sum_{m}\sum_{n} \hat{u}_{mn} \underbrace{\int_{-1}^1 \frac{\partial (L_m-L_{m+2})}{\partial x}\frac{\partial (L_k-L_{k+2})}{\partial x} {dx}}_{a_{km}} \underbrace{\frac{1}{2\pi}\int_{0}^{2\pi}  \exp(iny) \exp(-ily) {dy}}_{\delta_{ln}} \\
   &= a_{km} \delta_{ln} \hat{u}_{mn}
\end{align}
$$

Similarily for the second term

\begin{align}
\frac{1}{2\pi}\int_{-1}^1 \int_{0}^{2\pi} \frac{\partial u}{\partial y}\frac{\partial v^*}{\partial y} {dxdy} &= \frac{1}{2\pi}\int_{-1}^1 \int_{0}^{2\pi} \frac{\partial \sum_{m}\sum_{n} \hat{u}_{mn} \phi_{mn}}{\partial y}\frac{\partial \phi_{kl}^*}{\partial y }{dxdy}
\end{align}
\begin{align}
 &= \sum_{m}\sum_{n} \hat{u}_{mn} \underbrace{\int_{-1}^1 (L_m-L_{m+2})(L_k-L_{k+2}) {dx}}_{b_{km}} \underbrace{\frac{1}{2\pi}\int_{0}^{2\pi} l^2 \exp(iny) \exp(-ily){dy}}_{l^2\delta_{ln}} \\
   &= l^2 b_{km} \delta_{ln} \hat{u}_{mn}
\end{align}

which leads to:
$$
(\nabla u, \nabla v)_w = \left(a_{km} \delta_{ln} - l^2 b_{km} \delta_{ln}\right) \hat{u}_{mn}
$$

In [13]:
A = inner(grad(u), grad(v)) # <- list of two TPMatrices
print(A[0].mats)

[{0: array([ 6., 10., 14., 18., 22., 26.])}, {0: 1}]


# Complete Poisson solver with error verification in 1D

In [14]:
# Solve Poisson's equation
from sympy import symbols, sin, lambdify
from shenfun import * 

# Use sympy to compute manufactured solution
x = symbols("x")
ue = sin(4*np.pi*x)*(1-x**2) # `ue` is the manufactured solution
fe = ue.diff(x, 2) # `fe` is Poisson's right hand side for `ue`

SD = Basis(32, 'Chebyshev', bc=(0, 0))
u = TrialFunction(SD)
v = TestFunction(SD)

b = inner(v, Array(SD, buffer=fe)) # Array is initialized with `fe`
A = inner(v, div(grad(u)))

u_hat = A/b
print(u_hat.backward()-Array(SD, buffer=ue))

[ 3.72158258e-12 -3.70176702e-11  4.42706531e-11 -7.61924274e-11
  8.28891122e-11 -1.12129639e-10  1.17948290e-10 -1.43341068e-10
  1.48041746e-10 -1.68631331e-10  1.72095949e-10 -1.87183047e-10
  1.89420479e-10 -1.98592282e-10  1.99676609e-10 -2.02743711e-10
  2.02762585e-10 -1.99657624e-10  1.98611211e-10 -1.89401383e-10
  1.87200366e-10 -1.72079906e-10  1.68644987e-10 -1.48031087e-10
  1.43350110e-10 -1.17942406e-10  1.12133747e-10 -8.28862812e-11
  7.61939262e-11 -4.42697302e-11  3.70180041e-11 -3.72154551e-12]


# Complete (with MPI) in 3D with Fourier in 2 directions

In [15]:
from sympy import symbols, sin, cos, lambdify
from shenfun import *

# Use sympy to compute manufactured solution
x, y, z = symbols("x,y,z")
ue = (cos(4*x) + sin(2*y) + sin(4*z))*(1-x**2)
fe = ue.diff(x, 2) + ue.diff(y, 2) + ue.diff(z, 2)

C0 = Basis(32, 'Chebyshev', bc=(0, 0))
F1 = Basis(32, 'Fourier', dtype='D')
F2 = Basis(32, 'Fourier', dtype='d')
T = TensorProductSpace(comm, (C0, F1, F2))
u = TrialFunction(T)
v = TestFunction(T)

# Assemble left and right hand
f_hat = inner(v, Array(T, buffer=fe))
A = inner(v, div(grad(u)))

# Solve
solver = chebyshev.la.Helmholtz(*A)
u_hat = Function(T)
u_hat = solver(u_hat, f_hat)
assert np.linalg.norm(u_hat.backward()-Array(T, buffer=ue)) < 1e-12
print(u_hat.shape)

(32, 32, 17)


# Run with MPI distribution of arrays

Here we would normally run from a bash shell
<p style="margin-bottom:0.5cm;">

<div style="color:black"> <strong>[bash shell] mpirun -np 4 python poisson3D.py </strong> </div>

But since we are in a Jupyter notebook lets actually do this from python in a live cell:-)

In [16]:
import subprocess
subprocess.check_output('mpirun -np 4 python poisson3D.py', shell=True)

b'(32, 16, 8)\n(32, 16, 9)\n(32, 16, 8)\n(32, 16, 9)\n'

# Advanced topics

Coupled equations

# Fourier bases

<p style="margin-bottom:1cm;">

Especially interesting and efficient because they lead to diagonal matrices


In [17]:
C0 = Basis(N, 'Chebyshev', bc=(0, 0))
F0 = Basis(N, 'Fourier', dtype='D')
F1 = Basis(N, 'Fourier', dtype='d')
TP = TensorProductSpace(comm, (C0, F0, F1))

# MPI

MPI is handled under the hood by [`mpi4py-fft`](https://bitbucket.org/mpi4py/mpi4py-fft). Multidimensional arrays are distributed with no additional effort for the user.